In [1]:
from google.colab import drive
import os
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pwd
os.chdir("drive/MyDrive/Kolektif Öğrenme/project")
!pwd

/content
/content/drive/MyDrive/Kolektif Öğrenme/project


In [14]:
!pip install bert-for-tf2
!pip install sentencepiece
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 67.8 MB/s eta 0:00:00


In [15]:
import pandas as pd
from gensim.parsing.preprocessing import remove_stopwords
import re
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split, RepeatedKFold
import numpy as np
import nltk
from nltk.corpus import twitter_samples
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.layers import GlobalAveragePooling2D, Dense, MaxPooling2D, Flatten, Dropout, Input, Lambda
from keras.models import Model
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
import math

In [16]:
def cleaning(text):
    text = [txt.lower() for txt in re.findall(r"(?:^|(?<= ))[a-zA-Z]+(?= |$)", text)]
    text = remove_stopwords(" ".join(text))

    return "no text" if len(text) == 0 else text

In [17]:
nltk.download("twitter_samples")
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("omw-1.4")

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [35]:
df = pd.DataFrame(twitter_samples.strings("positive_tweets.json"), columns=["sentence"])
df2 = pd.DataFrame(twitter_samples.strings("negative_tweets.json"), columns=["sentence"])
df["result"] = 1
df2["result"] = 0
df = pd.concat([df, df2])

In [36]:
df.head()

,sentence,result
0,#FollowFriday @France_Inte @PKuchly57 @Milipol...,1
1,@Lamb2ja Hey James! How odd :/ Please call our...,1
2,@DespiteOfficial we had a listen last night :)...,1
3,@97sides CONGRATS :),1
4,yeaaaah yippppy!!! my accnt verified rqst has...,1


In [20]:
df["sentence"] = df["sentence"].apply(cleaning)
df = df[df["sentence"] != "no text"]

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9530 entries, 0 to 4999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  9530 non-null   object
 1   result    9530 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 223.4+ KB


In [22]:
import tensorflow_hub as hub
from transformers import BertTokenizer, TFBertModel

In [37]:
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1", trainable=True)

In [38]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased",do_lower_case=True)

def encoder(sentences):
  ids = []
  for sentence in sentences:
    encoding = tokenizer.encode_plus(
    sentence,
    max_length=16,
    truncation = True,
    add_special_tokens=True,
    return_token_type_ids=False,
    pad_to_max_length=True,
    return_attention_mask=False)
    ids.append(encoding["input_ids"])
  return ids

train_sents,test_sents, train_labels, test_labels  = train_test_split(df["sentence"],df["result"], test_size=0.2, stratify=df.result)

train_ids = encoder(train_sents)
test_ids = encoder(test_sents) 

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [40]:
train_ids = tf.convert_to_tensor(train_ids)
test_ids = tf.convert_to_tensor(test_ids)
test_labels = tf.convert_to_tensor(test_labels)
train_labels = tf.convert_to_tensor(train_labels)

In [41]:
max_len=500
bert_encoder = TFBertModel.from_pretrained("bert-base-uncased")
input_word_ids = Input(shape=(16,), dtype=tf.int32, name="input_word_ids")  
embedding = bert_encoder([input_word_ids])
dense = Lambda(lambda seq: seq[:, 0, :])(embedding[0])
dense = Dense(128, activation="relu",kernel_regularizer="l2")(dense)
output = Dense(1, activation="sigmoid",kernel_regularizer="l2")(dense)    

model = Model(inputs=[input_word_ids], outputs=output)  

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [42]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
train_history = model.fit(train_ids, train_labels, epochs = 5, batch_size = 64)

Epoch 1/5


125/125 [==============================] - 34s 199ms/step - loss: 1.9894 - accuracy: 0.5086
Epoch 2/5
125/125 [==============================] - 25s 202ms/step - loss: 1.0206 - accuracy: 0.4913
Epoch 3/5
125/125 [==============================] - 25s 203ms/step - loss: 0.8030 - accuracy: 0.4956
Epoch 4/5
125/125 [==============================] - 26s 208ms/step - loss: 0.7388 - accuracy: 0.4989
Epoch 5/5
125/125 [==============================] - 26s 208ms/step - loss: 0.7208 - accuracy: 0.5027


In [43]:
bert_encoder = Model(input_word_ids, model.get_layer(list(filter(lambda x: "lambda" in x.name,  model.layers))[0].name).output)
bert_encoder.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_word_ids (InputLayer)  [(None, 16)]             0         
                                                                 
 tf_bert_model_1 (TFBertMode  TFBaseModelOutputWithPoo  109482240
 l)                          lingAndCrossAttentions(l            
                             ast_hidden_state=(None,             
                             16, 768),                           
                              pooler_output=(None, 76            
                             8),                                 
                              past_key_values=None, h            
                             idden_states=None, atten            
                             tions=None, cross_attent            
                             ions=None)                          
                                                           

In [44]:
train_embed = bert_encoder.predict(train_ids)
test_embed = bert_encoder.predict(test_ids)

63/63 [==============================] - 5s 46ms/step


In [45]:
def generate_imp_space(X_train, y_train, X_test, imp_feature_size, foz):
    imp_train_data = X_train.values
    imp_test_data = X_test.values
    y_train = pd.DataFrame(y_train)
    d = len(X_train.columns)

    for i in range(0, imp_feature_size*foz):
        Xindis = np.random.permutation(d)
        for j in range(0, d-(foz-1), foz):
            sX = np.random.permutation(1)
            s1 = sX[0]

            s1data = X_train[X_train.index.isin(y_train[y_train == str(s1)].index)]
            s2data = X_train[~X_train.index.isin(y_train[y_train == str(s1)].index)]
            s1data = s1data.iloc[:, Xindis[j:j+(foz)]]
            s2data = s2data.iloc[:, Xindis[j:j+(foz)]]

            s1label = np.ones((s1data.values.shape[0], 1), dtype=int)
            s2label = -1*np.ones((s2data.values.shape[0], 1), dtype=int)
            Wdata = np.concatenate((s1data, s2data))

            Wdata = x2fx(Wdata)
            Wlabel = np.concatenate((s1label, s2label))
            W = np.matmul(np.matmul(np.linalg.pinv(
                np.matmul(Wdata.T, Wdata)), Wdata.T), Wlabel)

            WW = x2fx(X_train.iloc[:, Xindis[j:j+(foz)]].values)
            imp_train_data = np.concatenate(
                (imp_train_data, np.matmul(WW, W)), axis=1)

            TT = x2fx(X_test.iloc[:, Xindis[j:j+(foz)]].values)
            imp_test_data = np.concatenate(
                (imp_test_data, np.matmul(TT, W)), axis=1)

    return imp_train_data, imp_test_data


In [46]:
def x2fx(x, model="linear"):
    linear = np.c_[np.ones(x.shape[0]), x]
    if model == "linear":
        return linear
    if model == "purequadratic":
        return np.c_[linear, x**2]
    interaction = np.hstack([x[:, i]*x[:, j]
                            for i, j in comb(range(x.shape[1]), 2)]).T
    if model == "interaction":
        return np.c_[linear, interaction]
    if model == "quadratic":
        return np.c_[linear, interaction, x**2]

In [47]:
foz = 4
imp_feature_size = 1
n_estimators = 3
imp_acc = []
rfc_acc = []
estimators_imp = []
estimators = []
train_embed = pd.DataFrame(train_embed)
test_embed=pd.DataFrame(test_embed)
d = len(train_embed.columns)

for i in range(n_estimators):
    imp_tr, imp_ts = generate_imp_space(train_embed, train_labels, test_embed, imp_feature_size, foz)
    imp_d = imp_tr.shape[1]

    imp_sel_d = 2 * round(math.log2(imp_d))
    sel_d = 2*round(math.log2(d))

    imp_rfc = RandomForestClassifier(max_features=imp_sel_d, n_estimators=50, random_state=42)
    imp_rfc.fit(imp_tr, train_labels)
    estimators_imp.append(("imp_rfc"+str(i), imp_rfc))

    rfc = RandomForestClassifier(max_features=sel_d, n_estimators=50, random_state=42)
    rfc.fit(train_embed, train_labels)
    estimators.append(("rfc"+str(i), rfc))


voting_imp = VotingClassifier(estimators=estimators_imp)
voting_imp.fit(imp_tr, train_labels)

voting_rfc = VotingClassifier(estimators=estimators)
voting_rfc.fit(train_embed, train_labels)

imp_acc.append(voting_imp.score(imp_ts, test_labels))
rfc_acc.append(voting_rfc.score(test_embed, test_labels))


In [48]:
imp_acc

[0.6415]

In [49]:
rfc_acc

[0.648]

In [33]:
imp_acc

[0.5550891920251836]

In [34]:
rfc_acc

[0.55299055613851]